*Задание на лабораторную работу №3*

Целью данной работы является поиск движещуегося объекта на видео при неподвижной камере.

Необходимо:

- Обработать видео с целью лучшего распознавания объекта
- Создать маску фона
- Наложить маску на каждый кадр и выделить движущийся объект
- Выделить движущийся объект рамкой

# <font color='#F5DEB3'>1. Подключение библиотек, загрузка данных

In [7]:
import numpy as np
from time import time
import cv2
from IPython.display import clear_output

from google.colab import drive
from google.colab.patches import cv2_imshow

In [8]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# <font color='#F5DEB3'>2. Основная часть работы

Для выполнения задачи поиска движущегося объекта на видео с неподвижной камерой можно использовать метод вычитания фона. Этот метод позволяет создать маску фона, выделить движущиеся объекты и обвести их рамкой. Также отдельно выведим вид маски.

In [9]:
# Путь к видеофайлу на Google Drive
video_path = '/content/drive/MyDrive/Colab Notebooks/CV_LB3_data/video2.mp4'

# Создание объекта для чтения видео
cap = cv2.VideoCapture(video_path)

# Проверка, удалось ли открыть видео
if not cap.isOpened():
    raise IOError("Не удалось открыть видео.")

# Чтение первого кадра для создания маски фона
ret, first_frame = cap.read()
if not ret:
    raise IOError("Не удалось прочитать первый кадр.")

# Преобразование первого кадра в градации серого
first_frame_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
first_frame_gray = cv2.GaussianBlur(first_frame_gray, (5, 5), 0)  # Размытие для уменьшения шума

# Инициализация переменных для записи видео
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Создание объекта для записи результата
output_path_mask = '/content/drive/MyDrive/Colab Notebooks/CV_LB3_data/output_mask_video.mp4'
output_path = '/content/drive/MyDrive/Colab Notebooks/CV_LB3_data/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
out_mask = cv2.VideoWriter(output_path_mask, fourcc, fps, (frame_width, frame_height), isColor=False)

# Обработка каждого кадра
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Преобразование текущего кадра в градации серого
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)  # Размытие для уменьшения шума

    # Вычисление разницы между текущим кадром и фоном
    diff = cv2.absdiff(first_frame_gray, gray)

    # Бинаризация разницы для создания маски
    _, mask = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Морфологические операции для устранения шума
    mask = cv2.dilate(mask, None, iterations=2)
    mask = cv2.erode(mask, None, iterations=2)

    # Применение маски к текущему кадру
    masked_frame = cv2.bitwise_and(frame, frame, mask=mask)

    # Преобразование маскированного кадра в черно-белый формат
    masked_frame_gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)

    # Запись маскированного кадра в выходное видео
    out_mask.write(masked_frame_gray)

    # Поиск контуров на маске
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Обводка контуров и выделение движущихся объектов
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Фильтрация маленьких контуров
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Запись обработанного кадра в выходное видео
    out.write(frame)

    # Очищаем вывод
    clear_output(wait=True)

# Освобождение ресурсов
cap.release()
out.release()
out_mask.release()

print("Обработка завершена. Результаты смотрите в ", '/content/drive/MyDrive/Colab Notebooks/CV_LB3_data')

Обработка завершена. Результаты смотрите в  /content/drive/MyDrive/Colab Notebooks/CV_LB3_data
